In [33]:
#! /usr/bin/env python
# encoding: utf-8
#

import sys
import math
import re
from operator import attrgetter

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter, PDFPageAggregator
from pdfminer.layout import LAParams
from pdfminer.layout import LTTextContainer, LTTextBox
from pdfminer.pdfpage import PDFPage

import csv

In [4]:

DEBUG = False
QUOTE = False


# magick number
range_violation_col = (320,480)
range_first_col = ( 35, 179 )
range_column_last = (705, 790 )
#column_count = 6 # 列の数

In [6]:
def format_list_cell(node) :
    temp_str = node.get_text().rstrip("\n")

    # 違反した条文を列挙しているカラムの処理
    if "条" in temp_str :
        src_str = re.sub(r'(?<=(条|\d))\n(?!(の|$))', ',', temp_str)
    else:
        src_str = temp_str.replace("\n",',')
        

    str = src_str.replace("\n",'')
    return str


In [7]:
def remove_returncode(node) :
    src_str = node.get_text()
    
    if 'H' in src_str :
        src_str = re.sub(r'\n(?=H)', ',', src_str)
    

    str = src_str.replace("\n", '')
    

    if DEBUG :
        print(str)

    return str
         

In [8]:
rsrcmgr = PDFResourceManager()
laparams = LAParams()

#laparams.detect_vertical = True

device = PDFPageAggregator(rsrcmgr, laparams=laparams)

if DEBUG:
    print(sys.argv)

if ( len(sys.argv) > 1 ):
    filename = sys.argv[1]
else:
    exit(-1)

NameError: name 'p' is not defined

In [20]:
# 処理するPDFを開く
filename = 'hospital_list.pdf'
fp = open(filename, 'rb')
interpreter = PDFPageInterpreter(rsrcmgr, device)

parser = PDFParser(fp)
document = PDFDocument(parser, '')

In [28]:
print(parser)


dept_labor = "" # 都道府県労働局
last_modified_date = ""

header_text = ["企業・事業場名称", "所在地", "公表日", "違反法条", "事案概要", "その他参考事項"  ]

<PDFParser: <_io.BufferedReader name='hospital_list.pdf'>, bufpos=76736>


In [29]:
for page in PDFPage.create_pages(document):
    interpreter.process_page(page)
    # page.contents
    layout = device.get_result()

    if DEBUG :
        print(layout.pageid)

    if (layout.pageid == 1 ) :
        continue


    cells = list()
    
    
    for node in layout:
        if ( not issubclass(node.__class__ ,(LTTextBox, LTTextContainer ) ) ):
            
            #if DEBUG:
                #print(node)
            continue
        else:

            temp_str = node.get_text().rstrip("\n") 
            if temp_str.startswith("最終更新日") :
                last_modified_date = temp_str[6:]
                if DEBUG :
                    print(last_modified_date)
                continue
            if temp_str.endswith("労働局") :
                dept_labor = temp_str
                continue
            if temp_str.endswith("公表事案") :
                continue

            cells.append(node)


In [30]:
cells

[<LTTextBoxHorizontal(0) 312.050,1091.403,480.807,1102.443 '東京都肝臓専門医療機関指定リスト\n'>,
 <LTTextBoxHorizontal(1) 48.800,1061.348,97.280,1073.348 '指定番号\n'>,
 <LTTextBoxHorizontal(2) 188.340,1061.348,248.980,1073.348 '医療機関名\n'>,
 <LTTextBoxHorizontal(3) 45.760,1027.108,286.720,1039.108 'TK1321302 社会福祉法人緑風会\u3000緑風荘病院\n'>,
 <LTTextBoxHorizontal(4) 339.910,1054.628,364.070,1079.908 '郵便\n番号\n'>,
 <LTTextBoxHorizontal(5) 339.910,1020.548,364.150,1045.828 '189-\n0012\n'>,
 <LTTextBoxHorizontal(6) 509.370,1061.348,533.530,1073.348 '住所\n'>,
 <LTTextBoxHorizontal(7) 700.140,1061.348,724.300,1073.348 '電話\n'>,
 <LTTextBoxHorizontal(8) 371.270,1027.108,535.342,1039.108 '東村山市萩山町三丁目31番地1\n'>,
 <LTTextBoxHorizontal(9) 676.780,1027.153,747.740,1038.833 '042-392-1101\n'>,
 <LTTextBoxHorizontal(10) 45.760,985.348,286.720,997.348 'TK1321303 ワンズタワーすこやかクリニック\n'>,
 <LTTextBoxHorizontal(11) 339.910,978.788,364.150,1004.068 '189-\n0022\n'>,
 <LTTextBoxHorizontal(12) 371.270,978.788,480.618,1004.068 '東村山市野口町1-46\nワンズ

In [40]:
# 処理するPDFを開く
filename = 'hospital_list.pdf'
fp = open(filename, 'rb')
interpreter = PDFPageInterpreter(rsrcmgr, device)


parser = PDFParser(fp)
document = PDFDocument(parser, '')

dept_labor = "" # 都道府県労働局
last_modified_date = ""

header_text = ["企業・事業場名称", "所在地", "公表日", "違反法条", "事案概要", "その他参考事項"  ]

for page in PDFPage.create_pages(document):
    interpreter.process_page(page)
    # page.contents
    layout = device.get_result()

    if DEBUG :
        print(layout.pageid)

    if (layout.pageid == 1 ) :
        continue


    cells = list()


    for node in layout:
        if ( not issubclass(node.__class__ ,(LTTextBox, LTTextContainer ) ) ):
            
            #if DEBUG:
                #print(node)
            continue
        else:

            temp_str = node.get_text().rstrip("\n") 
            if temp_str.startswith("最終更新日") :
                last_modified_date = temp_str[6:]
                if DEBUG :
                    print(last_modified_date)
                continue
            if temp_str.endswith("労働局") :
                dept_labor = temp_str
                continue
            if temp_str.endswith("公表事案") :
                continue

            cells.append(node)

    filtered_cells = list(filter(lambda c: not(c.get_text().rstrip() in header_text) , cells))

    header_cells = list(filter(lambda c: c.get_text().rstrip() in header_text , cells))


    temp_cells = sorted(filtered_cells, key = attrgetter('x0'), reverse=False)
    sorted_cells = sorted(temp_cells, key = lambda c: (c.y0 + c.y1) // 20, reverse=True)
    #sorted_cells = sorted(temp_cells, key = attrgetter('y0'), reverse=True)
    #sorted_cells = sorted(temp_cells, key = attrgetter('x0'), reverse=False)

    
    columns = list()
    for cell in sorted_cells :
        if DEBUG:
            print(cell)

        center_x = ( cell.x0 + cell.x1 ) / 2.0
        #if ( center_x > range_first_col[0] and center_x < range_first_col[1] ) :
        if ( center_x > range_first_col[0] and cell.x0 < range_first_col[1] ) :
            # 先頭
            col_str = remove_returncode(cell)
            if ( cell.x1 < range_first_col[1] and ' ' in col_str ) :
                col_str = re.sub(' +', '　', col_str)
                print(col_str, file=sys.stderr)
           
            columns.insert(0, col_str)

        elif ( center_x > range_column_last[0] and center_x < range_column_last[1] ):
            # 最終カラム
            col_str = remove_returncode(cell)

            columns.append(col_str)


            # 出力

            # clean up
            purificated = list()
            for st in columns:
                if DEBUG:
                    print(st)

                if st.endswith("　") :
                    st = st.rstrip("　") # 末尾の全角空白を削除

                if re.search('  ',st):
                    # 半角空白の連続は全角スペースに
                    st = st.replace('  ',"\u3000")

                # 正常にパースできなかったセルを半角スペースで分割
                if (' ' in st ) and ( not '確定' in st ):
#                    temp = st.split(' ')
                    if re.search('(都|道|府|県).+(市|町|村)' ,st) :
                        if '条' in st :
                            # カラムが4つ結合しているケース
                            temp = st.rsplit(" ",3)
                        elif 'H' in st :
                            # カラムが3つ結合しているケース
                            temp = st.rsplit(" ",2)
                        else:
                            # カラムが2つ結合しているケース
                            temp = st.rsplit(" ",1)
                    else:
                        temp = re.split('(?<!）) ',st)

                    for ts in temp :
                        purificated.append(ts)
                else:
                    purificated.append(st)


            output = dept_labor + "\t" + last_modified_date + "\t" + "\t".join(purificated)
            if QUOTE:
                temp = list(map( lambda x: '"%s"' % x, output.split("\t") ))
                output = "\t".join(temp)
            

            print(output)
            
            columns = list()


        else:
            # それ以外
            if  center_x > range_violation_col[0]  and  center_x < range_violation_col[1]  :
                col_str = format_list_cell(cell)
            else:
                col_str = remove_returncode(cell)
            
            columns.append(col_str)
        with open('test.tsv', 'w') as f:
            w = csv.writer(f, delimiter='\t')
            w.writerows(output)


fp.close()
device.close()

		指定番号	東京都肝臓専門医療機関指定リスト	医療機関名	郵便,番号	住所	電話
		TK1310101	医療法人財団駿愛会　駿河台診療所	101-,0062	千代田区神田駿河台二丁目1番45号	03-3219-1121
		医療法人社団虎の門会霞が関アーバンクリニック	TK1310103	100-,0013	千代田区霞が関一丁目4番1号,日土地ビル1階	03-5157-3911
		TK1310104	医療法人社団　丸の内クリニック	100-,0005	千代田区丸の内一丁目6番2号,新丸の内センタービルディング4階	03-5220-2200
		TK1310105	医療法人社団亮友会　大塩医院	101-,0063	千代田区神田淡路町二丁目6番2号	03-3251-6878
		国家公務員共済組合連合会九段坂病院	TK1310107	102-,0074	千代田区九段南二丁目1番39号	03-3262-9191
		TK1310109	佐々木研究所附属　杏雲堂病院	101-,0062	千代田区神田駿河台一丁目8番地	03-3292-2051
		TK1310110	社会福祉法人　三井記念病院	101-,0024	千代田区神田和泉町1番地	03-3862-9111
		公益社団法人東京都教職員互助会三楽病院	TK1310111	101-,8326	千代田区神田駿河台二丁目5番地	03-3292-3981
		TK1310112	出版健康保険組合診療所	101-,0062	千代田区神田駿河台一丁目7番地	03-3292-5001
		TK1310113　日本大学病院	101-,0062	千代田区神田駿河台一丁目6番	03-3293-1711
		TK1310115　東京逓信病院	102-,8798	千代田区富士見二丁目14番23号	03-5214-7111
		TK1310117	医療法人財団医親会　海上ビル診療所	100-,0005	千代田区丸の内一丁目2番1号,東京海上日動ビルディング新館３階	03-3212-7690
		TK1310118	医療法人財団健貢会　東京クリニック	100-,0004	千代田区大手町二丁目2番1号,新大手町ビル	03-3516-7151
		医療法人社団T.O.P.ドクターズ東京国際クリニック	TK1310120	100-,6209	千

TK1310113　日本大学病院
TK1310115　東京逓信病院
TK1310213　高橋医院


		指定番号	東京都肝臓専門医療機関指定リスト	医療機関名	郵便,番号	住所	電話
		TK1310217	晴海３丁目クリニック	104-,0053	中央区晴海3-10-1　グラスシティ晴海1F	03-3536-8633
		TK1310218	日本橋ハートクリニック	103-,0021	中央区日本橋本石町3-2-12　社労士ビル2階	03-5201-8100
		TK1310301　今井内科医院	105-,0004	港区新橋五丁目7番12号	03-3432-2166
		TK1310304	国際医療福祉大学　三田病院	108-,8329	港区三田一丁目4番3号	03-3451-8121
		国家公務員共済組合連合会虎の門病院	TK1310305	105-,0001	港区虎ノ門二丁目2番2号	03-3588-1111
		TK1310306	社団法人北里研究所　北里研究所病院	108-,0072	港区白金五丁目9番1号	03-3444-6161
		独立行政法人地域治療機能推進機構東京高輪病院	TK1310307	108-,8606	港区高輪三丁目10番11号	03-3443-9191
		TK1310308	ふなきクリニック	107-,0052	港区赤坂三丁目15番9号,萬屋ビル5階	03-3505-0300
		TK1310309	東京大学医科学研究所附属病院	108-,8639	港区白金台四丁目6番1号	03-3443-8111
		TK1310310	東京慈恵会医科大学附属病院	105-,8471	港区西新橋三丁目19番18号	03-3433-1111
		TK1310311	東京都済生会中央病院	108-,0073	港区三田一丁目4番17号	03-3451-8211
		医療法人社団Veritas	Medical	Partners麻布医院	TK1310315	106-,0045	港区麻布十番一丁目11番1号	03-5545-8177
		医療法人財団順和会山王メディカルセンター	TK1310316	107-,0052	港区赤坂八丁目5番35号	03-3402-5581
		TK1310317	医療法人社団青友会　岡林医院	107-,0062	港区南青山五丁目1番18号　ボヌール青山1階	03-3498-5322
		医療法人健仁会ワールド・シティ益子クリニ

TK1310301　今井内科医院


		指定番号	東京都肝臓専門医療機関指定リスト	医療機関名	郵便,番号	住所	電話
		国立研究開発法人国立国際医療研究センタ－病院	TK1310405	162-,0052	新宿区戸山一丁目21番1号	03-3202-7181
		公益財団法人東京都保健医療公社大久保病院	TK1310406	160-,0021	新宿区歌舞伎町二丁目44番1号	03-5273-7711
		公益財団法人東京都予防医学協会保健会館クリニック	TK1310407	162-,0842	新宿区市谷砂土原町一丁目2番地	03-3269-1151
		TK1310408	社会福祉法人聖母会　聖母病院	161-,8521	新宿区中落合二丁目5番1号	03-3951-1111
		独立行政法人地域医療機能推進機構東京山手メディカルセンター	TK1310409	169-,0073	新宿区百人町三丁目22番1号	03-3364-0251
		TK1310410	東京医科大学病院	160-,0023	新宿区西新宿六丁目7番1号	03-3342-6111
		独立行政法人地域医療機能推進機構東京新宿メディカルセンター	TK1310411	162-,8543	新宿区津久戸町五丁目1番地	03-3269-8111
		TK1310412	東京女子医科大学病院	162-,8666	新宿区河田町8番1号	03-3353-8111
		東京都職員共済組合シティ・ホール診療所	TK1310413	163-,8001	新宿区西新宿二丁目8番1号,都庁第二本庁舎17階	03-5320-7358
		医療法人社団榊原厚生会新宿NSビルクリニック	TK1310416	163-,0804	新宿区西新宿二丁目4番1号NSビル4階	03-3343-3511
		TK1310417	あべメディカルクリニック	162-,0811	新宿区水道町4－29　宝ビル1階	03-6228-1783
		医療法人社団榊原厚生会新宿三井ビルクリニック	TK1310418	163-,0404	新宿区西新宿二丁目1番1号,新宿三井ビル4階	03-3344-3311
		TK1310419	いしい内科・外科クリニック	160-,0023	新宿区西新宿七丁目1番10号　守矢ビル5階	03-5937-3361
		TK1310420	目白よしみクリニック	16

...

TK1312328　佐藤医院


		指定番号	東京都肝臓専門医療機関指定リスト	医療機関名	郵便,番号	住所	電話
		TK1320103	東京医科大学八王子医療センタ－	193-,0998	八王子市館町1163番地	042-665-5611
		TK1320104	医療法人社団永生会　南多摩病院	193-,0832	八王子市散田町三丁目10番1号	042-663-0111
		医療法人社団めぐみ会南大沢メディカルプラザ	TK1320108	192-,0364	八王子市南大沢2-25	042-670-2460
		TK1320109	医療法人社団清智会　清智会記念病院	192-,0904	八王子市子安町三丁目24番15号	042-624-5111
		TK1320110	医療法人財団仁光会真宮病院	192-,0075	八王子市南新町23番地	042-625-0648
		医療法人社団志泉会富士森内科みなみのクリニック	TK1320113	192-,0917	八王子市西片倉3-1-21,第1みなみ野クリニックセンター2階	042-635-6711
		TK1320114	医療法人社団徳成会　八王子山王病院	192-,0042	八王子市中野山王2-15-16	042-626-1144
		医療法人社団　信隆会京王八王子クリニック	TK1320115	192-,0046	八王子市明神町4-7-14八王子ONビル	042-645-7878
		TK1320201	医療法人財団	川野病院	190-,0022	立川市錦町一丁目7番5号	042-522-8161
		TK1320202	国家公務員共済組合連合会　立川病院	190-,8531	立川市錦町四丁目2番22号	042-523-3131
		独立行政法人国立病院機構災害医療センター	TK1320203	190-,0014	立川市緑町3256番地	042-526-5511
		TK1320204	医療法人財団	立川中央病院	190-,0023	立川市柴崎町二丁目17番14号	042-522-7171
		TK1320301	かみやま内科クリニック	180-,0022	武蔵野市境一丁目9番9号	0422-60-3188
		TK1320302	武蔵野赤十字病院	180-,8610	武蔵野市境南町一丁目26番1号	0422-32-3111
		医療法人社団陽和

TK1320904　町田市民病院
TK1321102　公立昭和病院
TK1321202　日野市立病院


		指定番号	東京都肝臓専門医療機関指定リスト	医療機関名	郵便,番号	住所	電話
		TK1321302	社会福祉法人緑風会　緑風荘病院	189-,0012	東村山市萩山町三丁目31番地1	042-392-1101
		TK1321303	ワンズタワーすこやかクリニック	189-,0022	東村山市野口町1-46,ワンズタワー4F-2	042-399-0122
		公益財団法人東京都保健医療公社多摩北部医療センター	TK1321304	189-,8511	東村山市青葉町1-7-1	042-396-3811
		TK1321501	国立ふじみ内科	186-,0003	国立市富士見台二丁目45-12	042-571-9900
		TK1321801　公立福生病院	197-,8511	福生市加美平一丁目6番1号	042-551-1111
		TK1321802	医療法人社団幹人会　福生クリニック	197-,0012	福生市加美平三丁目35番13号	042-551-2312
		TK1321901	東京慈恵会医科大学附属第三病院	201-,0003	狛江市和泉本町四丁目11番1号	03-3480-1151
		TK1322001	社会医療法人財団大和会　東大和病院	207-,0014	東大和市南街一丁目13番12号	042-562-1411
		社会医療法人財団大和会東大和病院附属セントラルクリニック	TK1322002	207-,0014	東大和市南街２丁目3-1	042-562-5511
		TK1322101	独立行政法人国立病院機構　東京病院	204-,0023	清瀬市竹丘三丁目1番1号	042-491-2111
		社会医療法人財団大和会武蔵村山病院	TK1322301	208-,0022	武蔵村山市榎一丁目1番5号	042-566-3111
		TK1322401	井上内科クリニック	206-,0034	多摩市鶴牧二丁目24番12号	042-372-3660
		医療法人社団めぐみ会田村クリニック	TK1322402	206-,0033	多摩市落合一丁目32番1号,ペペリビル4階	042-356-0677
		TK1322403	日本医科大学多摩永山病院	206-,0025	多摩市永山一丁目7番1号	042-371-2111
		公益財団法人東京都保健医療公社

TK1321801　公立福生病院
TK1322502　稲城市立病院
